# Mine Production Planning with Stockpiling (with Gurobi)

## Introduction
Open-pit mine production scheduling is a central planning task in mining operations. The goal is to decide **when and how much material to extract and process** from different blocks of the mine, over a finite time horizon, in order to maximize the economic value of the project. Traditionally, this is formulated as the **Open Pit Mine Production Scheduling Problem (OPMPSP)**, which already is a large-scale mixed-integer optimization problem due to precedence constraints among blocks, mining and processing capacity limits, and economic discounting.

In practice, mining operations often make use of **stockpiles**: mined ore that is not immediately processed can be stored and reclaimed later. This flexibility allows the mine to adapt to fluctuations in ore quality and market conditions, increasing overall Net Present Value (NPV). However, the introduction of stockpiles complicates the optimization model: once material is placed into a stockpile, it is mixed homogeneously with existing material, and when it is reclaimed, the quality corresponds to the **average composition** of the stockpile. Mathematically, this leads to **bilinear (quadratic) constraints**, which are computationally challenging for standard MILP solvers.

The paper by [Bley, Boland, Froyland, and Zuckerberg (2012)](https://web.maths.unsw.edu.au/~froyland/bbfz.pdf) develops advanced formulations (Natural, Aggregated, and Discretized models) to handle stockpile effects within an optimization framework. Their results show that carefully designed extended formulations and branching strategies yield much tighter relaxations and practical solvability compared to naive models.


## Problem Description
We consider the **Open Pit Mine Production Scheduling Problem with Stockpiling (OPMPSP+S).**
The complete problem formulation and context are provided in the attached paper. 
Here, we focus on the implementation and include only brief explanatory comments.

In [50]:
%load_ext autoreload
%autoreload 2

import gurobipy as gp
from gurobipy import GRB, quicksum
from itertools import product
from gurobipy import multidict, tuplelist
import pandas as pd
import numpy as np
from graphviz import Digraph
from optutils import gurobi_helpers, plotting
from optutils.plotting import draw_problem
from optutils.gurobi_helpers import (define_variables, define_objective, delta, define_constraints)
from optutils.utils import read_txt, parse_arcs_head_format, td_to_df, _print_solution
import pandas as pd
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load and Process Data

In [4]:
# read parameters_marvin.txt and extract its informations
parameters = read_txt("../data/mine_production_planning/parameters_marvin.txt")

# q
q = parameters["annual_discount"]

# time_periods
time_periods = parameters["time_periods"]
time_periods = list(range(1, time_periods+1))

# mining capacity and processing capacity
mining_capacity, processing_capacity = parameters["capacity_mining"], parameters["capacity_processing"]

# time_periods, mining capacity ,processing capacity in multidict format
time_periods, mining_capacity, processing_capacity = multidict({
    t: [mining_capacity, processing_capacity]
    for t in range(1, len(time_periods)+1)
})

# scalar costs / global (time-invariant)
m = parameters["cost_mining"]                 # mining cost per tonne of rock
proc_cost = parameters["cost_processing"]     # processing cost per tonne of ore
c1 = parameters["price_metal"]                # sales price per tonne of metal
c2 = parameters["price_metal2"]               # sales price2 per tonne of metal

In [6]:
# import blocks with [rock_tonnage, ore_tonnage, metal_tonnage], i.e. [R, O, A]
df = pd.read_csv("../data/mine_production_planning/panels.csv", sep=";")
df.columns = df.columns.str.strip()           # to avoid whitespace

# drop summary rows (min, max, avg, std-dev)
df = df[pd.to_numeric(df["block"], errors="coerce").notna()].copy()
df["block"] = df["block"].astype(int)

# ensure numeric types of the columns
df[["block", "rock", "ore", "metal"]] = df[["block", "rock", "ore", "metal"]].apply(pd.to_numeric, errors="coerce")
df["block"] = df["block"].astype(int)

# removes any row where at least one of the columns "rock", "ore", or "metal" is missing (NaN).
df.dropna(subset=["rock", "ore", "metal"], how="any", inplace=True)   

# Build multidict: blocks, R, O, A
blocks, R, O, A = multidict({
    int(b): [float(r), float(o), float(m)]
    for b, r, o, m in zip(df["block"], df["rock"], df["ore"], df["metal"])
})

In [8]:
# Define plants for processing to extract the metal from the rock
plants = ["P"]

# Define stockpiles 
stockpiles = ["S"]

# Define waste dumps
waste_dumps = ["W"]

In [10]:
# import precedence constraints from arcs.txt (which i must be mined before j)
#precedence = read_txt("../data/mine_production_planning/arcs.txt")
with open("../data/mine_production_planning/arcs.txt", encoding='utf-8') as f:
    arcs = [line.strip() for line in f]
    
P = parse_arcs_head_format(arcs, blocks)

## I- Aggregate Tracking

### (1) Aggregate Tracking Model (AT)

$$
\begin{aligned}
S_{\mathrm{AT}}
:= \Big\{ & (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
          \in [0,1]^{6(N\times T)} \times \mathbb{R}_{\ge 0}^{4T} \times [0,1]^T : \\
& (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
   \text{ satisfies } (1)\text{--}(10),\ (12)\text{--}(18), \\
& x \in \{0,1\}^{N\times T} \Big\}.
\end{aligned}
$$

In [14]:
model_AT = gp.Model('MineProductionPlanningAT_real')

# set global parameters
model_AT.params.nonConvex = 2

Set parameter Username
Set parameter LicenseID to value 2588857
Academic license - for non-commercial use only - expires 2025-11-22
Set parameter NonConvex to value 2


In [16]:
# Define decision variables with define_variables with x is set to binary
x, y, z_p, z_s, oreSP, oreS_rem, metalSP, metalS_rem, z_ss, z_sp, f_t = define_variables(model_AT,
                                                                                            blocks,
                                                                                            time_periods,
                                                                                            plants,
                                                                                            stockpiles,
                                                                                            waste_dumps,
                                                                                            x_binary = True,
                                                                                            aggregate = True
                                                                                           )

# Define objective function with define_objective
define_objective(model_AT, y, z_p, oreSP, metalSP, blocks, plants, stockpiles, time_periods, O, A, R, c1, proc_cost, m, q)

# Define constraints with define_constraints with set enforce_mixing=False
define_constraints(model_AT, x, y, z_p, z_s,
                      oreS_rem, metalS_rem, oreSP, metalSP,
                      blocks, plants, stockpiles, time_periods,
                      O, A, R, P,mining_capacity,processing_capacity,
                      enforce_mixing=False,aggregate=True, enforce_mixing_aggregate=True,
                      z_ss=z_ss, z_sp=z_sp, f_t=f_t)

In [18]:
model_AT.write('../models/MineProductionPlanningAT_real_gurobi.lp')    # model saved

model_AT.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2

Optimize a model with 7807 rows, 8755 columns and 56573 nonzeros
Model fingerprint: 0x661b2240
Model has 1445 quadratic constraints
Variable types: 7310 continuous, 1445 integer (1445 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-05, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 1271 rows and 1396 columns
Presolve time: 0.16s
Presolved: 18333 rows, 9906 columns, 73711 nonzeros
Presolved model has 2631 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 8786 continuous, 1120 integer (1120 binar

In [66]:
_print_solution(model_AT, aggregate=True, y=y, z_p=z_p, z_s=z_s, tol=1e-9)

,Block,Period,Mine,Plant,Proc,Stock,Pile
,1,11,0.914202,P,0.914202,-,0.0
,1,12,0.085798,P,0.085798,-,0.0
,2,6,1.000000,P,1.000000,-,0.0
,3,2,1.000000,P,1.000000,-,0.0
,4,1,0.685054,P,0.685054,-,0.0
...,...,...,...,...,...,...,...
,81,8,0.556521,P,0.556521,-,0.0
,82,6,1.000000,P,1.000000,-,0.0
,83,3,1.000000,P,1.000000,-,0.0
,84,5,0.979095,P,0.979095,-,0.0


### (2) Aggregate Tracking Model (AT-IP)

$$
\begin{aligned}
S_{\mathrm{AT}\text{-}\mathrm{IP}}
:= \Big\{ & (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
          \in [0,1]^{6(N\times T)} \times \mathbb{R}_{\ge 0}^{4T} \times [0,1]^T : \\
& \big([x], y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f\big)
   \text{ satisfies } (1)\text{--}(10),\ (12)\text{--}(17), \\
& x \in \{0,1\}^{N\times T} \Big\}.
\end{aligned}
$$

In [22]:
model_AT_IP = gp.Model('MineProductionPlanningAT-IP-real')

# set global parameters
model_AT_IP.params.nonConvex = 2

Set parameter NonConvex to value 2


In [24]:
# Define decision variables with define_variables with x is set to binary
x, y, z_p, z_s, oreSP, oreS_rem, metalSP, metalS_rem, z_ss, z_sp, f_t = define_variables(model_AT_IP,
                                                                                            blocks,
                                                                                            time_periods,
                                                                                            plants,
                                                                                            stockpiles,
                                                                                            waste_dumps,
                                                                                            x_binary = True,
                                                                                            aggregate = True
                                                                                           )

# Define objective function with define_objective
define_objective(model_AT_IP, y, z_p, oreSP, metalSP, blocks, plants, stockpiles, time_periods, O, A, R, c1, proc_cost, m, q)

# Define constraints with define_constraints with set enforce_mixing=False
define_constraints(model_AT_IP, x, y, z_p, z_s,
                      oreS_rem, metalS_rem, oreSP, metalSP,
                      blocks, plants, stockpiles, time_periods,
                      O, A, R, P,mining_capacity,processing_capacity,
                      enforce_mixing=False,aggregate=True, enforce_mixing_aggregate=False,
                      z_ss=z_ss, z_sp=z_sp, f_t=f_t)

In [26]:
model_AT_IP.write('../models/MineProductionPlanningAT_IP_real_gurobi.lp')    # model saved

model_AT_IP.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2

Optimize a model with 7807 rows, 8755 columns and 56573 nonzeros
Model fingerprint: 0x96cfd5b8
Variable types: 7310 continuous, 1445 integer (1445 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [1e-05, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 1379 rows and 1629 columns
Presolve time: 0.15s
Presolved: 6428 rows, 7126 columns, 48344 nonzeros
Variable types: 6006 continuous, 1120 integer (1120 binary)

Root relaxation: objective 1.565672e+08, 3617 iterations, 0.08 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |

In [68]:
_print_solution(model_AT_IP, aggregate=True, y=y, z_p=z_p, z_s=z_s, tol=1e-9)

,Block,Period,Mine,Plant,Proc,Stock,Pile
,1,11,0.914202,P,0.914202,-,0.0
,1,12,0.085798,P,0.085798,-,0.0
,2,6,1.000000,P,1.000000,-,0.0
,3,2,1.000000,P,1.000000,-,0.0
,4,1,0.685054,P,0.685054,-,0.0
...,...,...,...,...,...,...,...
,81,8,0.556521,P,0.556521,-,0.0
,82,6,1.000000,P,1.000000,-,0.0
,83,3,1.000000,P,1.000000,-,0.0
,84,5,0.979095,P,0.979095,-,0.0


### (3) Aggregate Tracking Model (AT-LP)

$$
\begin{aligned}
S_{\mathrm{AT}\text{-}\mathrm{LP}}
:= \Big\{ & (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
          \in [0,1]^{6(N\times T)} \times \mathbb{R}_{\ge 0}^{4T} \times [0,1]^T : \\
& (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
   \text{ satisfies } (1)\text{--}(10),\ (12)\text{--}(17) \Big\}.
\end{aligned}
$$

In [30]:
model_AT_LP = gp.Model('MineProductionPlanningAT-LP-real')

# set global parameters
#model_AT_LP.params.nonConvex = 2

In [32]:
# Define decision variables with define_variables with x is set to continuous
x, y, z_p, z_s, oreSP, oreS_rem, metalSP, metalS_rem, z_ss, z_sp, f_t = define_variables(model_AT_LP,
                                                                                            blocks,
                                                                                            time_periods,
                                                                                            plants,
                                                                                            stockpiles,
                                                                                            waste_dumps,
                                                                                            x_binary = False,
                                                                                            aggregate = True
                                                                                           )

# Define objective function with define_objective
define_objective(model_AT_LP, y, z_p, oreSP, metalSP, blocks, plants, stockpiles, time_periods, O, A, R, c1, proc_cost, m, q)

# Define constraints with define_constraints with set enforce_mixing=False
define_constraints(model_AT_LP, x, y, z_p, z_s,
                      oreS_rem, metalS_rem, oreSP, metalSP,
                      blocks, plants, stockpiles, time_periods,
                      O, A, R, P,mining_capacity,processing_capacity,
                      enforce_mixing=False,aggregate=True, enforce_mixing_aggregate=False,
                      z_ss=z_ss, z_sp=z_sp, f_t=f_t)

In [34]:
model_AT_LP.write('../models/MineProductionPlanningAT_LP_real_gurobi.lp')    # model saved

model_AT_LP.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7807 rows, 8755 columns and 56573 nonzeros
Model fingerprint: 0xae164708
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [1e-05, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+07]
Presolve removed 1253 rows and 1451 columns
Presolve time: 0.04s
Presolved: 6554 rows, 7304 columns, 50087 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.03s

Barrier statistics:
 AA' NZ     : 1.468e+05
 Factor NZ  : 6.116e+05 (roughly 10 MB of memory)
 Factor Ops : 8.996e+07 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.15 seconds (0.07 work units)
Barrier solve interrupted - model solved by an

In [64]:
_print_solution(model_AT_LP, aggregate=True, y=y, z_p=z_p, z_s=z_s, tol=1e-9)

,Block,Period,Mine,Plant,Proc,Stock,Pile
,1,11,0.914202,P,0.914202,-,0.0
,1,12,0.085798,P,0.085798,-,0.0
,2,6,1.000000,P,1.000000,-,0.0
,3,2,1.000000,P,1.000000,-,0.0
,4,1,0.685054,P,0.685054,-,0.0
...,...,...,...,...,...,...,...
,81,8,0.556521,P,0.556521,-,0.0
,82,6,1.000000,P,1.000000,-,0.0
,83,3,1.000000,P,1.000000,-,0.0
,84,5,0.979095,P,0.979095,-,0.0


#### Achieved improvement 

In [76]:
obj_AT = model_AT.ObjVal
obj_AT_IP = model_AT_IP.ObjVal
obj_AT_LP = model_AT_LP.ObjVal

In [82]:
obj_AT_IP - obj_AT 

17163.235770106316

In [84]:
obj_AT_LP - obj_AT_IP

14092.690235495567

In [86]:
obj_AT_LP - obj_AT

31255.926005601883

## II- Discretised out-fraction formulations